In [ ]:
pip install langchain_google_genai
pip install pypdf
pip install langchain_community
pip install faiss-cpu

In [ ]:
import os
from langchain_core.prompts import ChatPromptTemplate
# Standard parser to convert the LLM's message object into a clean string
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from google import genai
from google.colab import userdata
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
from google.colab import userdata
geminikey=userdata.get('test')

In [ ]:
client = genai.Client(api_key=geminikey)

In [ ]:
for m in client.models.list():
  if 'embedContent' in m.supported_actions:
    print(m.name)

models/gemini-embedding-001


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
gemini_embed = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001",api_key=geminikey)

In [ ]:
loader = PyPDFLoader("/content/AboutNJ.pdf")
docs = loader.load()

In [ ]:
chunks = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

In [ ]:
vectorstore = FAISS.from_documents(chunks, gemini_embed)

In [ ]:
vectorstore.save_local("LangFAISS")

In [ ]:
db = FAISS.load_local("LangFAISS", gemini_embed,allow_dangerous_deserialization=True)

In [ ]:
retriever = db.as_retriever(k=1)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Context: {context}\n\nQuestion: {question}\n\nAnswer:"
)

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",api_key=geminikey)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
answer = chain.invoke("Provide me professional summary about Naresh Jani")
print(answer)

Naresh Jani has over 14 years of experience in delivering client-focused solutions across diverse domains. He is proficient in Oracle Technologies, including Oracle BIP Reports and OIC. Additionally, he possesses cutting-edge skills in Machine Learning, Data Science, and Generative AI, coupled with strong analytical capabilities to interpret data and provide valuable insights. He has a proven track record of collaborating with business functional areas to research and identify optimal technical solutions for improving business processes.
